In [1]:
import torch
from wt import DWT1DForward, DWT1DInverse, DWT2DForward, DWT2DInverse
from tft.transforms import WPT1D, WPT2D, IWPT1D, IWPT2D

wt1d = DWT1DForward(J=1, mode='periodization', wave='bior4.4')
wpt1d = WPT1D(wt=wt1d, J=8)
iwt1d = DWT1DInverse(mode='periodization', wave='bior4.4')
iwpt1d = IWPT1D(iwt=iwt1d, J=8)

wt2d = DWT2DForward(J=1, mode='periodization', wave='bior4.4')
wpt2d = WPT2D(wt=wt2d, J=4)
iwt2d = DWT2DInverse(mode='periodization', wave='bior4.4')
iwpt2d = IWPT2D(iwt=iwt2d, J=4)

b = 4; c = 12; h = 384; w = 256; ℓ = 49152
x1d = torch.randn(b, c, ℓ)
x2d = torch.randn(b, c, h, w)

with torch.no_grad():
    X1d = wpt1d(x1d)
    xhat1d = iwpt1d(X1d)
    X2d = wpt2d(x2d)
    xhat2d = iwpt2d(X2d)

assert (xhat1d - x1d).abs().max() < 1e-5
assert (xhat2d - x2d).abs().max() < 1e-5